In [21]:
# define logging and working directory
import os
# define GPU id to use
# 0 = 1080 Bus ID 2
# 1 = Titan Bus ID 131
# 2 = Titan Bus ID 132
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
current_gpu = '/device:GPU:0'


import logging
import json
import platform
import SimpleITK as sitk
import glob
import datetime
import random
from collections import Counter
import matplotlib.pyplot as plt
import keras
from keras.preprocessing.image import ImageDataGenerator
from medpy.metric.binary import hd, dc,jc,precision,recall
import tensorflow as tf
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from ipywidgets import interact
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# setup working directory
print(os.getcwd())
# change working directory to project root - neccessary for jupyter notebooks
if platform.system() is "Windows":
    os.chdir('E:\\sven\\git\\cardio')
else:
    os.chdir('/Users/minority/Code/Git/cardio')
print(os.getcwd())


from src.utils.utils_io import Console_and_file_logger, ensure_dir, save_config
from src.utils.myshow import myshow, myshow3d
from src.visualization.visualize import plot_3d_vol, plot_4d_vol
from src.data.dataset import get_metadata_maybe, filter_4d_vol, describe_sitk
from src.data.generators import DataGenerator3D
from src.data.dataset import get_3d_img_msk_files
from src.utils.my_metrics import jaccard_coef, jaccard_coef_background, jaccard_coef_rv, jaccard_coef_lv, jaccard_coef_myo, bce_dice_iou_loss, weighted_categorical_crossentropy, cce_dice_loss, weighted_cce_dice_coef
from src.utils.unet_3d_metrics import weighted_dice_coefficient_loss
from src.models.ModelManager import create_3D_unet
from src.utils.KerasCallbacks import TrainValTensorBoard, CustomImageWriter3D



# define experiment name for report, model and log paths + filenames
EXPERIMENT = '3D_unet_cce_dice_loss'
now = datetime.datetime.now()

# image params
ARCHITECTURE = '3D'
IMG_Z = 10
IMG_WIDTH = 256
IMG_HEIGHT = 256
SPACING = (1.25,1.25,8)
IMG_CHANNELS = 1
MASK_VALUES = [0, 1, 2, 3]  
MASK_CLASSES = len(MASK_VALUES)
# Background = 0 = Y[:,:,0]
# RV = 1 = Y[:,:,1] 
# Myo = 2 = Y[:,:,2] 
# LV = 3 = Y[:,:,3]


# path params
DATASET = 'tetra'  # 'acdc' # or 'tetra'
TRAIN_SMALL = 'data/raw/tetra/small/train/'
TRAIN_PATH = 'data/raw/tetra/3D/train/'
VAL_PATH = 'data/raw/tetra/3D/val/'
TEST_PATH = 'data/raw/tetra/3D/test/'

CREATE_DATASET = False

MODEL_PATH = os.path.join(
    os.path.join('models', EXPERIMENT), str(now.strftime("%Y-%m-%d_%H_%M")))
TENSORBOARD_LOG_DIR = os.path.join(
    os.path.join('reports/tensorboard_logs', EXPERIMENT),
    str(now.strftime("%Y-%m-%d_%H_%M")))
CONFIG_PATH = os.path.join(os.path.join('reports/configs/',EXPERIMENT),
                           str(now.strftime("%Y-%m-%d_%H_%M")))
HISTORY_PATH = os.path.join(os.path.join('reports/history/',EXPERIMENT),
                            str(now.strftime("%Y-%m-%d_%H_%M")))
# training params
seed = 42
BATCHSIZE = 1  # 64, 16, 1
EPOCHS = 5
AUGMENT = False
SHUFFLE = True
EPOCHS_BETWEEN_CHECKPOINTS = 5
OPTIMIZER = 'adam'  # adam, sgd, softmax # https://keras.io/optimizers/
ACTIVATION = 'elu'  # 'elu' --> works well with binary_crossentropy and bce_dice_loss, relu does not work, it clips negative values, bse does return negative values
LEARNING_RATE = 0.01  # is not used in the model at the moment
MONITOR_FUNCTION = 'val_jaccard_coef'
MONITOR_MODE = 'max'
#LOSS_FUNCTION = keras.losses.categorical_crossentropy
metrics = [jaccard_coef]

weights = np.array([1,1,1,1]) # Class one at 1, class 2, 10 times the normal weights, class 3 and 4 20x.
LOSS_FUNCTION = weighted_categorical_crossentropy(weights)
#LOSS_FUNCTION = cce_dice_loss


Console_and_file_logger(EXPERIMENT, logging.INFO)
ensure_dir(TENSORBOARD_LOG_DIR)
ensure_dir(MODEL_PATH)
ensure_dir(CONFIG_PATH)
ensure_dir(HISTORY_PATH)
config = save_config(locals())

# set warnings lvl for skimage
#warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
# define a Tensorflow config
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
tf_session = tf.Session(config=tf_config)
tf.keras.backend.set_session(tf_session)

2019-05-03 17:33:11,264 INFO config saved:
 {
    "ACTIVATION": "elu",
    "ARCHITECTURE": "3D",
    "AUGMENT": false,
    "BATCHSIZE": 1,
    "CONFIG_PATH": "reports/configs/3D_unet_cce_dice_loss\\2019-05-03_17_33",
    "CREATE_DATASET": false,
    "DATASET": "tetra",
    "EPOCHS": 5,
    "EPOCHS_BETWEEN_CHECKPOINTS": 5,
    "EXPERIMENT": "3D_unet_cce_dice_loss",
    "HISTORY_PATH": "reports/history/3D_unet_cce_dice_loss\\2019-05-03_17_33",
    "IMG_CHANNELS": 1,
    "IMG_HEIGHT": 256,
    "IMG_WIDTH": 256,
    "IMG_Z": 10,
    "LEARNING_RATE": 0.01,
    "MASK_CLASSES": 4,
    "MASK_VALUES": [
        0,
        1,
        2,
        3
    ],
    "MODEL_PATH": "models\\3D_unet_cce_dice_loss\\2019-05-03_17_33",
    "MONITOR_FUNCTION": "val_jaccard_coef",
    "MONITOR_MODE": "max",
    "OPTIMIZER": "adam",
    "SHUFFLE": true,
    "TENSORBOARD_LOG_DIR": "reports/tensorboard_logs\\3D_unet_cce_dice_loss\\2019-05-03_17_33",
    "TEST_PATH": "data/raw/tetra/3D/test/",
    "TRAIN_PATH": "dat

E:\sven\git\cardio
E:\sven\git\cardio


## Get training, val and test-files

In [22]:
x_train_f, y_train_f = get_3d_img_msk_files('data/raw/tetra/3D/train/')
x_val_f, y_val_f = get_3d_img_msk_files('data/raw/tetra/3D/val/')
x_test_f, y_test_f = get_3d_img_msk_files('data/raw/tetra/3D/test/')

logging.info('x_train files: {}, y_train files: {}'.format(len(x_train_f), len(y_train_f)))
logging.info('x_val files: {}, y_val files: {}'.format(len(x_val_f), len(y_val_f)))
logging.info('x_test files: {}, y_test files: {}'.format(len(x_test_f), len(y_test_f)))

2019-05-03 17:33:12,824 INFO x_train files: 757, y_train files: 757
2019-05-03 17:33:12,839 INFO x_val files: 126, y_val files: 126
2019-05-03 17:33:12,839 INFO x_test files: 127, y_test files: 127


## Create Datagenerator

In [23]:
# create a batch generator, filter size, for faster testing
SPACING = (2.4,2.2,8) # test spacing to get images with shaoe 128, 128, 128
config['IMG_Z'] = 16
config['IMG_WIDTH'] = 128
config['IMG_HEIGHT'] = 128
batch_generator = DataGenerator3D(x_train_f, y_train_f, batch_size=config['BATCHSIZE'], dim=(config['IMG_Z'], config['IMG_WIDTH'], config['IMG_HEIGHT']), spacing=SPACING, mask_values=MASK_VALUES, grey=True)
validation_generator = DataGenerator3D(x_val_f, y_val_f, batch_size=config['BATCHSIZE'], dim=(config['IMG_Z'], config['IMG_WIDTH'], config['IMG_HEIGHT']), spacing=SPACING, mask_values=MASK_VALUES, grey=True)
test_generator = DataGenerator3D(x_test_f, y_test_f, dim=(config['IMG_Z'], config['IMG_WIDTH'], config['IMG_HEIGHT']), spacing=SPACING, mask_values=MASK_VALUES)


2019-05-03 17:33:14,384 INFO Create DataGenerator
2019-05-03 17:33:14,384 INFO No augmentation
2019-05-03 17:33:14,399 INFO Create DataGenerator
2019-05-03 17:33:14,399 INFO No augmentation
2019-05-03 17:33:14,399 INFO Create DataGenerator
2019-05-03 17:33:14,399 INFO No augmentation


## Visualize one batch

In [24]:
@interact
def visualize_batchgenerator(batch=(0,len(batch_generator), 1)):
    # plot x and y
    x, y = batch_generator.__getitem__(batch)
    if x[0].shape[-1]>4: # make shure to work with channel first batches
        x = np.swapaxes(x,1,-1)
        y = np.swapaxes(y,1,-1)
    plot_3d_vol(x[0], y[0],fig_size=(40,20))
    nda = (y[0]).astype(np.float64)
    nda_cat = nda[...,1:] # ignore background for plotting
    myshow3d(sitk.GetImageFromArray(nda_cat))
    
    nda_img = (x[0]).astype(np.float64)
    nda_img = nda_img[...,0] # ignore background for plotting
    myshow3d(sitk.GetImageFromArray(nda_img))

interactive(children=(IntSlider(value=378, description='batch', max=757), Output()), _dom_classes=('widget-int…

## Create Model

In [17]:
def get_callbacks():
    
    callbacks=[]
    ensure_dir(MODEL_PATH)

        
    def feed_inputs_4_tensorboard():
        """
        Returns some sample images for visualisation in Tensorboard
        """
        feed = {}

        # build our feed dict for later tensorboard visualisation
        #feed['train_lower'] = get_tetra_slice_examples(TRAIN_PATH, examples=4, volume_part='lower')
        #feed['val_lower'] = get_tetra_slice_examples(VAL_PATH, examples=2, volume_part='lower')
        
        #feed['train_middle'] = get_tetra_slice_examples(TRAIN_PATH, examples=4, volume_part='middle')
        #feed['val_middle'] = get_tetra_slice_examples(VAL_PATH, examples=2, volume_part='middle')
        
        #feed['train_upper'] = get_tetra_slice_examples(TRAIN_PATH, examples=4, volume_part='upper')
        #feed['val_upper'] = get_tetra_slice_examples(VAL_PATH, examples=2, volume_part='upper')
        
        # test the batch- and validation-generator
        x_t, y_t = batch_generator.__getitem__(0)
        x_v, y_v = validation_generator.__getitem__(0)
        # get the first 2 elements from both batches
        x_t, y_t = x_t[0:2], y_t[0:2]
        x_v, y_v = x_v[0:2], y_v[0:2]
        # build our feed dict for later tensorboard visualisation
        feed['train_generator'] = (x_t, y_t)
        feed['val_generator'] = (x_v, y_v)
        logging.info('feed 4 Tensorboard is ready')
        return feed
    

    #callbacks.append(IoU(validation_generator, len(MASK_VALUES), EPOCHS_BETWEEN_CHECKPOINTS))
    callbacks.append(CustomImageWriter3D(log_dir = config['TENSORBOARD_LOG_DIR'],image_freq = config['EPOCHS_BETWEEN_CHECKPOINTS'], feed_inputs_4_display = feed_inputs_4_tensorboard()))
    callbacks.append(keras.callbacks.EarlyStopping(patience=10, verbose=1, monitor=MONITOR_FUNCTION, mode=MONITOR_MODE))
    #callbacks.append(WeightsSaver(MODEL_PATH, model_freq = 2))
    callbacks.append(keras.callbacks.ModelCheckpoint(os.path.join(MODEL_PATH, 'checkpoint.h5'), verbose=1, save_best_only=True, monitor=MONITOR_FUNCTION, mode=MONITOR_MODE))
    callbacks.append(TrainValTensorBoard(log_dir=config['TENSORBOARD_LOG_DIR'], batch_size=config['BATCHSIZE']))
    callbacks.append(keras.callbacks.TerminateOnNaN())
    return callbacks
    

In [18]:
from src.models.Isensee2017 import isensee2017_model

In [1]:

input_ = (128, 128, 16, 4)
model = isensee2017_model(input_shape=input_, 
                          n_base_filters=16, 
                          depth=5, 
                          dropout_rate=0.3,
                          n_segmentation_levels=1, 
                          n_labels=4, 
                          initial_learning_rate=5e-4,
                          activation_name="sigmoid")
model.summary()

NameError: name 'isensee2017_model' is not defined

In [20]:
config['EPOCHS'] = 100
initial_epoch = 0

# training

with tf.device(current_gpu):
    # fit model with trainingsgenerator
    logging.info('Fit model')
    results = model.fit_generator(
        generator=batch_generator,
        epochs=config['EPOCHS'],
        callbacks=get_callbacks(),
        validation_data=validation_generator,
        initial_epoch=initial_epoch,
        max_queue_size=10,
        workers=4,
        verbose=1)

2019-05-02 23:30:14,693 INFO Fit model
2019-05-02 23:30:14,833 INFO feed 4 Tensorboard is ready


Epoch 1/100
757/757 [==============================] - 468s 618ms/step - loss: 0.0444 - dice_coefficient: nan - categorical_accuracy: 0.9658 - val_loss: 4.7684e-07 - val_dice_coefficient: nan - val_categorical_accuracy: 0.9696


E:\sven\git\cardio\src\visualization\visualize.py:60: RuntimeWarning: invalid value encountered in greater
  y_ = (y_> 0.5).astype(np.float32)
C:\Users\s.koehler\AppData\Local\conda\conda\envs\sv\lib\site-packages\keras\callbacks.py:526: RuntimeWarning: Early stopping conditioned on metric `val_jaccard_coef` which is not available. Available metrics are: val_loss,val_dice_coefficient,val_categorical_accuracy,loss,dice_coefficient,categorical_accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
C:\Users\s.koehler\AppData\Local\conda\conda\envs\sv\lib\site-packages\keras\callbacks.py:435: RuntimeWarning: Can save best model only with val_jaccard_coef available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


Epoch 2/100
757/757 [==============================] - 460s 608ms/step - loss: 4.7684e-07 - dice_coefficient: nan - categorical_accuracy: 0.9693 - val_loss: 4.7684e-07 - val_dice_coefficient: nan - val_categorical_accuracy: 0.9696
Epoch 3/100
757/757 [==============================] - 459s 606ms/step - loss: 4.7684e-07 - dice_coefficient: nan - categorical_accuracy: 0.9693 - val_loss: 4.7684e-07 - val_dice_coefficient: nan - val_categorical_accuracy: 0.9696
Epoch 4/100
757/757 [==============================] - 458s 605ms/step - loss: 4.7684e-07 - dice_coefficient: nan - categorical_accuracy: 0.9693 - val_loss: 4.7684e-07 - val_dice_coefficient: nan - val_categorical_accuracy: 0.9696
Epoch 5/100
757/757 [==============================] - 459s 606ms/step - loss: 4.7684e-07 - dice_coefficient: nan - categorical_accuracy: 0.9693 - val_loss: 4.7684e-07 - val_dice_coefficient: nan - val_categorical_accuracy: 0.9696
Epoch 6/100
757/757 [==============================] - 462s 610ms/step - los

KeyboardInterrupt: 

## Write trainings history to disk

In [ ]:
logging.info(results.history)
df_history = pd.DataFrame(results.history)

df_history.to_csv(os.path.join('reports/history/3D_unet/', config['EXPERIMENT'] + '.csv'))
df_history.plot()

## Save model to disk

In [9]:
from src.utils.utils_io import ensure_dir
model_json = model.to_json()
model_path = os.path.join('models/', config['EXPERIMENT'])
ensure_dir(model_path)
with open(os.path.join(model_path, 'model.json'), "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
name = 'weights_e-{0}_val_loss-{1}.h5'.format(config['EXPERIMENT'], '01')
model.save_weights(os.path.join(model_path, name))
logging.info("Saved model to disk: {}".format(model_path))

2019-04-25 17:18:04,686 INFO Saved model to disk: models/3D_unet_cce_dice_loss


## Calculate metrics

In [13]:
# predict for all batches
from medpy.metric.binary import hd, dc,jc,precision,recall
import numpy as np
gt_ = []
pred_ = []
img_ = []
n_batches = len(test_generator)
#n_batches = 2
logging.info('load and predict {} batches'.format(n_batches))
#with tf.device(current_gpu):
#    pred = model.predict_generator(batch_generator, steps = len(batch_generator))
#
#gt_ = [gt_.extend(batch[1]) for batch in batch_generator]
    


for idx,batch in enumerate(test_generator):
    if idx <= n_batches:
        with tf.device(current_gpu):
            img_.extend(batch[0])
            pred_.extend(((model.predict_on_batch(batch[0]))>=0.5).astype(np.bool))
        gt_.extend(batch[1])
    else:
        break

# reshape
gt = np.array(gt_)
pred = np.array(pred_)

logging.info('gt shape: {}'.format(gt.shape))
logging.info('pred shape: {}'.format(pred.shape))
del gt_
del pred_

2019-04-25 18:35:04,100 INFO load and predict 127 batches
2019-04-25 18:36:15,094 INFO gt shape: (128, 12, 256, 256, 4)
2019-04-25 18:36:15,094 INFO pred shape: (128, 12, 256, 256, 4)


In [14]:
# calc medpy scores
jaccard_coef = jc(pred, gt)
dice_score = dc(pred, gt)
precision_score = precision(pred, gt)
recall_score = recall(pred, gt)

logging.info('jac: {}'.format(jaccard_coef))
logging.info('dice: {}'.format(dice_score))
logging.info('prec: {}'.format(precision_score))
logging.info('recall: {}'.format(recall_score))

2019-04-25 18:36:25,956 INFO jac: 0.9857375262342911
2019-04-25 18:36:25,956 INFO dice: 0.9928175433171392
2019-04-25 18:36:25,956 INFO prec: 0.9925043322609612
2019-04-25 18:36:25,956 INFO recall: 0.9931309521198273


In [15]:
# calc IOU per channel
for c in range(pred.shape[-1]):
    pred_ = pred[...,c]
    gt_ = gt[...,c]
    # calc medpy scores
    jaccard_coef = jc(pred_, gt_)
    dice_score = dc(pred_, gt_)
    precision_score = precision(pred_, gt_)
    recall_score = recall(pred_, gt_)

    logging.info('jac: {}'.format(jaccard_coef))
    logging.info('dice: {}'.format(dice_score))
    logging.info('prec: {}'.format(precision_score))
    logging.info('recall: {}'.format(recall_score))



2019-04-25 18:36:30,169 INFO jac: 0.9944598505959397
2019-04-25 18:36:30,169 INFO dice: 0.9972222306694191
2019-04-25 18:36:30,169 INFO prec: 0.9978699238022684
2019-04-25 18:36:30,169 INFO recall: 0.9965753777949433
2019-04-25 18:36:34,300 INFO jac: 0.8077829767147708
2019-04-25 18:36:34,300 INFO dice: 0.8936725117112567
2019-04-25 18:36:34,300 INFO prec: 0.8813529914136737
2019-04-25 18:36:34,300 INFO recall: 0.9063413179348874
2019-04-25 18:36:38,247 INFO jac: 0.6981590059794022
2019-04-25 18:36:38,247 INFO dice: 0.8222539862534762
2019-04-25 18:36:38,247 INFO prec: 0.7661480602238935
2019-04-25 18:36:38,247 INFO recall: 0.887226639436396
2019-04-25 18:36:42,303 INFO jac: 0.8724628853017204
2019-04-25 18:36:42,303 INFO dice: 0.9318880413067686
2019-04-25 18:36:42,303 INFO prec: 0.9350864580785502
2019-04-25 18:36:42,303 INFO recall: 0.9287114300017919


In [ ]:
from src.utils.my_metrics import jaccard_coef, jaccard_coef_background, jaccard_coef_rv, jaccard_coef_lv, jaccard_coef_myo, bce_dice_iou_loss, weighted_categorical_crossentropy, cce_dice_loss, weighted_cce_dice_coef

In [12]:
# calc IOU per channel
for c in range(pred.shape[-1]):
    pred_ = pred[...,c]
    gt_ = gt[...,c]
    # calc medpy scores
    jaccard_coef = jc(pred_, gt_)
    dice_score = dc(pred_, gt_)
    precision_score = precision(pred_, gt_)
    recall_score = recall(pred_, gt_)

    logging.info('jac: {}'.format(jaccard_coef))
    logging.info('dice: {}'.format(dice_score))
    logging.info('prec: {}'.format(precision_score))
    logging.info('recall: {}'.format(recall_score))


2019-04-25 17:22:30,732 INFO jac: 0.9962716483214747
2019-04-25 17:22:30,732 INFO dice: 0.9981323425187849
2019-04-25 17:22:30,732 INFO prec: 0.9989544550326821
2019-04-25 17:22:30,732 INFO recall: 0.9973115820449483
2019-04-25 17:22:55,880 INFO jac: 0.8690431935156842
2019-04-25 17:22:55,880 INFO dice: 0.929933772029107
2019-04-25 17:22:55,880 INFO prec: 0.907100853645496
2019-04-25 17:22:55,880 INFO recall: 0.9539458401729277
2019-04-25 17:23:19,842 INFO jac: 0.747106838663043
2019-04-25 17:23:19,842 INFO dice: 0.8552503168435417
2019-04-25 17:23:19,842 INFO prec: 0.7930340435369907
2019-04-25 17:23:19,858 INFO recall: 0.9280598319179684
2019-04-25 17:23:43,617 INFO jac: 0.9157806465379281
2019-04-25 17:23:43,617 INFO dice: 0.9560391459146079
2019-04-25 17:23:43,617 INFO prec: 0.9670873491505381
2019-04-25 17:23:43,617 INFO recall: 0.9452405252532905


In [ ]:
from src.visualization.visualize import plot_3d_vol

In [10]:
plot_3d_vol(img_[0], gt[0])

NameError: name 'img_' is not defined

In [ ]:
plot_3d_vol(img_[0], pred[0])

In [ ]:
print(pred_[0].shape)

In [ ]:
plot_3d_vol(pred_[0])

In [19]:
pred[0].max()

1.0